In [69]:
import advent
import numpy as np
data = advent.get_lines(17)[0]
data = [{'>': True, '<': False}[c] for c in data]

In [70]:
# Basically my dumb version of an enum
pieces = ['D', 'T', 'J', 'I', 'O']

# convenience function for moving x/y coord:
d = lambda t: (t[0], t[1]-1)
r = lambda t: (t[0]+1, t[1])
l = lambda t: (t[0]-1, t[1])
u = lambda t: (t[0], t[1]+1)
tadd = lambda l, r: (l[0] + r[0], l[1] + r[1])

def highest_rock(board):
    # New rocks should spawn at y = highest_rock + 4
    if np.sum(board) == 0: return -1
    last_occurence = np.argmax((board.sum(axis=0)>0)[::-1])
    return board.shape[1] - last_occurence - 1

def start_position(piece, board):
    # The 'start' and the 'anchor' position is defined as the BOTTOM-LEFT piece
    # in case of the T-piece, this anchor will be empty! (but still the anchor)
    return (2, highest_rock(board) + 4)

def fall(board, piece, anchor):
    # Return boolean: if the fall was succesful
    # does not check if the piece was already clear, only if the new spaces are clear
    # new_anchor = (anchor[0], anchor[1] - 1) # only if this function returns True
    if anchor[1] == 0: return False # reached the floor
    if piece == 'D': return not (board[d(anchor)] or board[tadd(anchor, (1, -1))] or board[tadd(anchor, (2, -1))] or board[tadd(anchor, (3, -1))])
    if piece == 'T': return not (board[anchor] or board[d(r(anchor))] or board[r(r(anchor))])
    if piece == 'J': return not (board[d(anchor)] or board[d(r(anchor))] or board[d(r(r(anchor)))])
    if piece == 'I': return not board[d(anchor)]
    if piece == 'O': return not (board[d(anchor)] or board[d(r(anchor))])

def gust(board, direction, piece, anchor):
    # Returns true if gust is 'succesfull'/unblocked. direction = True if right, False if left
    if not direction and anchor[0] == 0: return False
    if direction and anchor[0] == {'D': 3, 'T': 4, 'J': 4, 'I': 6, 'O': 5}[piece]: return False
    if piece == 'D':
        if direction: return not board[tadd(anchor, (0, 4))]
        return not board[l(anchor)]
    if piece == 'T':
        if direction: return not (board[tadd(anchor, (2, 0))] or board[tadd(anchor, (3, 1))] or board[tadd(anchor, (2, 2))])
        return not (board[anchor] or board[tadd(anchor, (-1, 1))] or board[tadd(anchor, (0, 2))])
    if piece == 'J':
        if direction: return not (board[tadd(anchor, (3, 0))] or board[tadd(anchor, (3, 1))] or board[tadd(anchor, (3, 2))])
        return not (board[tadd(anchor, (-1, 0))] or board[tadd(anchor, (1, 1))] or board[tadd(anchor, (1, 2))])
    if piece == 'I':
        if direction: return not (board[tadd(anchor, (1, 0))] or board[tadd(anchor, (1, 1))] or board[tadd(anchor, (1, 2))] or board[tadd(anchor, (1, 3))])
        return not (board[tadd(anchor, (-1, 0))] or board[tadd(anchor, (-1, 1))] or board[tadd(anchor, (-1, 2))] or board[tadd(anchor, (-1, 3))])
    if piece == 'O':
        if direction: return not (board[tadd(anchor, (2, 0))] or board[tadd(anchor, (2, 1))])
        return not (board[tadd(anchor, (-1, 0))] or board[tadd(anchor, (-1, 1))])
    
def save(board, piece, anchor):
    # saves in place!!!!
    if piece == 'D': board[anchor[0] : (anchor[0] + 4), anchor[1]] = 1
    if piece == 'T':
        board[anchor[0] : (anchor[0] + 3), anchor[1] + 1] = 1
        board[anchor[0] + 1, anchor[1] : (anchor[1] + 3)] = 1
    if piece == 'J':
        board[anchor[0] : (anchor[0] + 3), anchor[1]] = 1
        board[anchor[0] + 2, anchor[1] : (anchor[1] + 3)] = 1
    if piece == 'I': board[anchor[0], anchor[1] : (anchor[1] + 4)] = 1
    if piece == 'O':
        board[anchor[0] : (anchor[0] + 2), anchor[1]] = 1
        board[anchor[0] : (anchor[0] + 2), anchor[1] + 1] = 1
    return board
    

In [71]:
def full_fall(board, piece, gust_position):

    anchor = start_position(piece, board)
    # First gust, then fall, until the fall returns False
    while True:
        direction = data[gust_position]
        if gust(board, direction, piece, anchor): anchor = r(anchor) if direction else l(anchor)
        gust_position = (gust_position + 1) % len(data)

        if fall(board, piece, anchor): anchor = d(anchor)
        else: break

    save(board, piece, anchor)
    return (board, gust_position)


In [73]:
next_piece = {'D': 'T', 'T': 'J', 'J': 'I', 'I': 'O', 'O': 'D'}

def full_fall_n_times(board, n=2022, piece='D', gust_position=0):
    for _ in range(n):
        board, gust_position = full_fall(board, piece, gust_position)
        piece = next_piece[piece]
    return board, piece, gust_position


board = np.zeros((7, 5000))

board, _, _ = full_fall_n_times(board)

print(highest_rock(board) + 1)
# 3194 is wrong? but it works on the example input :(


3194
